# Cài đặt các thư viện cần thiết để cào dữ liệu

In [1]:
%pip install requests-html beautifulsoup4 pandas numpy lxml lxml_html_clean nest_asyncio pyppeteer-py3 pyppeteer requests selenium cloudscraper selenium webdriver-manager


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement pyppeteer-py3 (from versions: none)
ERROR: No matching distribution found for pyppeteer-py3
You should consider upgrading via the 'c:\Program Files (x86)\Microsoft Visual Studio\Shared\Python39_64\python.exe -m pip install --upgrade pip' command.


In [2]:
import requests
import datetime
import pandas as pd 
import numpy as np
from requests_html import HTML
from requests_html import HTMLSession
from bs4 import BeautifulSoup

# Cào link

In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import pandas as pd
import time

# Đường dẫn tới ChromeDriver
# chromedriver_path = 'C:\\chromedriver\\chromedriver.exe'

# # Cấu hình ChromeDriver service
# service = Service(chromedriver_path)

# # Khởi tạo WebDriver cho Cốc Cốc
# options = webdriver.ChromeOptions()
# options.add_argument("--headless")  # Chạy ẩn danh (không hiển thị trình duyệt)

try:
    
    # Danh sách để lưu trữ các link
    arr = []

    # Lặp qua các trang
    for page in range(1, 10):
        driver = webdriver.Chrome()
        url = f"https://www.nhatot.com/mua-ban-can-ho-chung-cu?page={page}"
        print(f"Đang lấy dữ liệu từ trang: {url}")
        # Mở trang web
        driver.get(url)
        # Lấy tất cả các đường link sản phẩm
        links = driver.find_elements(By.CSS_SELECTOR,"a.AdItem_adItem__gDDQT")
        for link in links:
            arr.append(link.get_attribute('href'))
        print(f"Số lượng link đã lấy: {len(arr)}")
        print("-" * 50)
        # Đóng trình duyệt
        driver.quit()
except Exception as e:
    print(f"Có lỗi xảy ra: {str(e)}")
# Tạo DataFrame từ danh sách các link
df = pd.DataFrame(arr, columns=['links'])
# Lưu DataFrame vào file CSV
df.to_csv('data\\chotot.csv', index=False)
print("Hoàn thành lấy dữ liệu!")


Đang lấy dữ liệu từ trang: https://www.nhatot.com/mua-ban-can-ho-chung-cu?page=1
Số lượng link đã lấy: 25
--------------------------------------------------
Đang lấy dữ liệu từ trang: https://www.nhatot.com/mua-ban-can-ho-chung-cu?page=2
Số lượng link đã lấy: 50
--------------------------------------------------
Hoàn thành lấy dữ liệu!


# Test thử xem lấy được dữ liệu không

In [4]:
frame = pd.read_csv('data\\chotot.csv')
frame.head(20)
frame.info

<bound method DataFrame.info of                                                 links
0   https://www.nhatot.com/mua-ban-can-ho-chung-cu...
1   https://www.nhatot.com/mua-ban-can-ho-chung-cu...
2   https://www.nhatot.com/mua-ban-can-ho-chung-cu...
3   https://www.nhatot.com/mua-ban-can-ho-chung-cu...
4   https://www.nhatot.com/mua-ban-can-ho-chung-cu...
5   https://www.nhatot.com/mua-ban-can-ho-chung-cu...
6   https://www.nhatot.com/mua-ban-can-ho-chung-cu...
7   https://www.nhatot.com/mua-ban-can-ho-chung-cu...
8   https://www.nhatot.com/mua-ban-can-ho-chung-cu...
9   https://www.nhatot.com/mua-ban-can-ho-chung-cu...
10  https://www.nhatot.com/mua-ban-can-ho-chung-cu...
11  https://www.nhatot.com/mua-ban-can-ho-chung-cu...
12  https://www.nhatot.com/mua-ban-can-ho-chung-cu...
13  https://www.nhatot.com/mua-ban-can-ho-chung-cu...
14  https://www.nhatot.com/mua-ban-can-ho-chung-cu...
15  https://www.nhatot.com/mua-ban-can-ho-chung-cu...
16  https://www.nhatot.com/mua-ban-can-ho-chung-cu

# Cào dữ liệu

In [5]:
address=[]
estate_condition,price_m2, main_direction,floor,id_house=[],[],[],[],[]
balcony_direction,Legal_papers,Character,acreage,bedroom,block=[],[],[],[],[],[]
bath_room,types,furniture_condition=[],[],[]
person,price=[],[]
items =['Tình trạng bất động sản:','Diện tích:','Giá/m2:','Số phòng ngủ:','Tên phân khu/Lô/Block/Tháp:','Tầng số:',
       'Số phòng vệ sinh:','Loại hình căn hộ:','Giấy tờ pháp lý:','Mã căn','Tình trạng nội thất:','Hướng cửa chính:',
       'Hướng ban công','Đặc điểm căn hộ']

In [6]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from requests_html import HTML

def getdata(link):
    # Cấu hình Selenium WebDriver
    options = Options()
    # options.add_argument("--headless")  # Chạy trình duyệt ở chế độ headless (không hiển thị giao diện)
    options.add_argument("--disable-gpu")  # Tắt GPU (không cần thiết cho chế độ headless)
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    # Tạo instance của trình điều khiển Chrome
    driver = webdriver.Chrome(options=options)
        # Mở trang web        
    driver.get(link)
        # Lấy nội dung HTML của trang
    html_text = driver.page_source
    driver.quit()  # Đóng trình duyệt sau khi lấy dữ liệu
        # Phân tích cú pháp HTML để trích xuất dữ liệu
    r_html = HTML(html=html_text)
    if len(r_html.find(".notfound-message")) == 0:
        arr = []
        # col-md-8 là chứa toàn bộ web
        r_table = r_html.find(".col-md-8")
        if len(r_table) > 0:
            # test là toàn bộ mã html của trang đang truy xuất
            test = r_table[0]
            inf = test.find('.col-xs-12')
            attribute=test.find(".AdParam_adParamContainerVeh__Vz4Zt")
            price = test.find('.AdDecriptionVeh_price__u_N83')
            # temp = r_html.find('a')
            # col = temp[6].find('b')
            info = inf[0].text.replace("Xem bản đồ","").strip() #xóa chữ bản đồ ở phía cuối
            arr.append(info)  # dia chi nha
            arr.append(attribute[0].text)  # thong tin can nha
            arr.append(price[0].text)  # gia nha
        else:
            return []

        str1 = arr
        str1 = [arr]
        temp = arr[1].split('\n')
        idx = []
        idx.append([arr[0].split('\n')[0]])
        for i in temp:
            idx.append(i.split('\n'))
            idx.append([arr[2]])
        return idx
    else:
        return []




In [7]:
count=0
for link in frame['links']:
    print(link)
    data = getdata(link)
    dem =0 
    if (len(data)>0):
        address+=data[0]
        for item in items:
            mang=[]
            flag=0
            for i in data:
                if item in i[0]:
                    mang.append(i[0].split(':')[1])
                    flag = 1
                    break
            dem = dem+1
            if flag==1:
                if (dem ==1): estate_condition+=mang
                elif (dem ==2): acreage+=mang
                elif (dem ==3): price_m2+=mang
                elif (dem ==4): bedroom+=mang
                elif (dem ==5): block+=mang
                elif (dem ==6): floor+=mang
                elif (dem ==7): bath_room+=mang
                elif (dem ==8): types+=mang
                elif (dem ==9): Legal_papers+=mang
                elif (dem ==10): id_house+=mang
                elif (dem ==11): furniture_condition+=mang
                elif (dem ==12): main_direction+=mang
                elif (dem ==13): balcony_direction+=mang
                elif (dem ==14): Character+=mang
            else:
                if (dem ==1): estate_condition.append(np.nan)
                elif (dem ==2): acreage.append(np.nan)
                elif (dem ==3): price_m2.append(np.nan)
                elif (dem ==4): bedroom.append(np.nan)
                elif (dem ==5): block.append(np.nan)
                elif (dem ==6): floor.append(np.nan)
                elif (dem ==7): bath_room.append(np.nan)
                elif (dem ==8): types.append(np.nan)
                elif (dem ==9): Legal_papers.append(np.nan)
                elif (dem ==10): id_house.append(np.nan)
                elif (dem ==11): furniture_condition.append(np.nan)
                elif (dem ==12): main_direction.append(np.nan)
                elif (dem ==13): balcony_direction.append(np.nan)
                elif (dem ==14): Character.append(np.nan)  
        price+=data[len(data)-1]

https://www.nhatot.com/mua-ban-can-ho-chung-cu-quan-son-tra-da-nang/117289273.htm#px=SR-stickyad-[PO-1][PL-top]
https://www.nhatot.com/mua-ban-can-ho-chung-cu-thanh-pho-thu-duc-tp-ho-chi-minh/117040353.htm#px=SR-stickyad-[PO-2][PL-top]
https://www.nhatot.com/mua-ban-can-ho-chung-cu-quan-10-tp-ho-chi-minh/117346827.htm#px=SR-stickyad-[PO-3][PL-top]
https://www.nhatot.com/mua-ban-can-ho-chung-cu-thanh-pho-tay-ninh-tay-ninh/116943923.htm#px=SR-stickyad-[PO-4][PL-top]
https://www.nhatot.com/mua-ban-can-ho-chung-cu-thanh-pho-di-an-binh-duong/117282681.htm#px=SR-stickyad-[PO-5][PL-top]
https://www.nhatot.com/mua-ban-can-ho-chung-cu-quan-tan-phu-tp-ho-chi-minh/116481833.htm
https://www.nhatot.com/mua-ban-can-ho-chung-cu-thanh-pho-di-an-binh-duong/117338873.htm#px=SR-special_display_ad-[PO-7][PL-default]
https://www.nhatot.com/mua-ban-can-ho-chung-cu-huyen-nha-be-tp-ho-chi-minh/117309995.htm
https://www.nhatot.com/mua-ban-can-ho-chung-cu-thanh-pho-thu-duc-tp-ho-chi-minh/115673324.htm
https://w

In [8]:
df = {
        'DiaChi' : address,
        'TinhTrangBDS' : estate_condition,
        'DienTich' : acreage,
        'Gia/m2'   : price_m2,
        'Phongngu': bedroom,
      'TenPhanKhu': block,
      'SoTang': floor,
      'PhongTam':bath_room,
      'Loai': types,
      'GiayTo': Legal_papers,
      'MaCanHo': id_house,
      'TinhTrangNoiThat': furniture_condition,
      'HuongCuaChinh': main_direction,
      'HuongBanCong': balcony_direction,
      'DacDiem':Character,
      'Gia': price,
}
df = pd.DataFrame(df)
df.to_csv('raw_data.csv',encoding="utf-8-sig",index=False)
df

,DiaChi,TinhTrangBDS,DienTich,Gia/m2,Phongngu,TenPhanKhu,SoTang,PhongTam,Loai,GiayTo,MaCanHo,TinhTrangNoiThat,HuongCuaChinh,HuongBanCong,DacDiem,Gia
0,"Đường Trần Hưng Đạo, Phường An Hải Tây, Quận S...",Chưa bàn giao,70 m²,"20,57 triệu/m²",2 phòng,NaN,NaN,2 phòng,Chung cư,NaN,NaN,NaN,Bắc,Đông Nam,NaN,"1,44 tỷ - 70 m2đ"
1,"Đường Võ Chí Công, Phường Thạnh Mỹ Lợi (Quận 2...",Chưa bàn giao,62 m²,"25,81 triệu/m²",2 phòng,NaN,NaN,1 phòng,Chung cư,NaN,NaN,NaN,Tây Bắc,Đông Nam,NaN,"1,6 tỷ - 62 m2đ"
2,"258, Đường Lý Thường Kiệt, Phường 14, Quận 10,...",Chưa bàn giao,80 m²,85 triệu/m²,2 phòng,C,12,2 phòng,NaN,NaN,NaN,NaN,Đông Nam,NaN,NaN,"6,8 tỷ - 80 m2đ"
3,"Đường Trần Quốc Toản, Phường 2, Thành phố Tây ...",Chưa bàn giao,61 m²,"17,21 triệu/m²",2 phòng,A2,NaN,2 phòng,Chung cư,Sổ hồng riêng,NaN,NaN,NaN,NaN,NaN,"1,05 tỷ - 61 m2đ"
4,"Xa lộ Hà Nội, phường Bình Thắng, tp Dĩ An Bình...",Chưa bàn giao,59 m²,"30,51 triệu/m²",2 phòng,A-B,NaN,2 phòng,Chung cư,NaN,NaN,NaN,NaN,Đông Nam,NaN,"1,8 tỷ - 59 m2đ"
5,"số 88, N1, Phường Sơn Kỳ, Quận Tân Phú, Tp Hồ ...",Đã bàn giao,107 m²,"51,87 triệu/m²",3 phòng,NaN,NaN,2 phòng,Chung cư,NaN,NaN,NaN,Tây Bắc,Đông Nam,NaN,"5,55 tỷ - 107 m2đ"
6,"Tân Đông Hiệp, Phường Tân Đông Hiệp, Thành phố...",Đã bàn giao,72.4 m²,"21,13 triệu/m²",3 phòng,charm plaza,13,1 phòng,NaN,NaN,NaN,NaN,Đông,NaN,NaN,"1,53 tỷ - 72.4 m2đ"
7,"Nguyễn Hữu Thọ, Xã Phước Kiển, Huyện Nhà Bè, T...",Chưa bàn giao,70 m²,"45,71 triệu/m²",2 phòng,NaN,NaN,2 phòng,Chung cư,Hợp đồng mua bán,NaN,NaN,NaN,NaN,NaN,"3,2 tỷ - 70 m2đ"
8,"Căn hộ B2-11.11, Đường số 11, Phường Tam Bình ...",Đã bàn giao,119 m²,"30,67 triệu/m²",3 phòng,A,12,NaN,Chung cư,NaN,NaN,NaN,Đông,NaN,NaN,"3,65 tỷ - 119 m2đ"
9,"Đường Nguyễn Ái Quốc, Phường Hoá An, Thành phố...",Đã bàn giao,55.5999 m²,"13,49 triệu/m²",2 phòng,NaN,3,1 phòng,Chung cư,Sổ hồng riêng,NaN,NaN,NaN,NaN,NaN,750 triệu - 55.5.999 m2đ
